# CELLxGENE Analysis by Dataset

First, load relevant packages.

In [1]:
# standard imports
import numpy as np
import pandas as pd
import os
import time

# path manipulation
from pathlib import Path

# string manipulation
import re

# single cell analysis
import scanpy as sc
import anndata

# import project config file
import sys
sys.path.append('../../..')
import project_config

Note that directories are relative to the project path.

In [2]:
# set directories
neuroKG_dir = project_config.PROJECT_DIR / 'Data' / 'NeuroKG'
ensembl_dir = project_config.PROJECT_DIR / 'Data' / 'Ensembl'
data_dir = project_config.PROJECT_DIR / 'Data' / 'CELLxGENE'
cellxgene_dir = Path("/n/data1/hms/dbmi/zitnik/lab/datasets/2023-05-CELLxGENE/census")
cellxgene_data = Path("/n/data1/hms/dbmi/zitnik/lab/users/an252/NeuroKG/neuroKG/Data/CELLxGENE")

Read protein coding genes from Ensembl, see `Data/Ensembl/README.md` for documentation on data retrieval steps.

In [3]:
# read protein coding genes and fix names
mart = pd.read_csv(ensembl_dir / 'mart_export.txt')
new_mart_names = [x.lower() for x in mart.columns.values]
new_mart_names = [re.sub('\\s\\([^)]+\\)', '', x) for x in new_mart_names]
new_mart_names = [re.sub('\\s+', '_', x) for x in new_mart_names]
mart.columns = new_mart_names

Filter for protein coding genes that also have Entrez IDs.

In [4]:
# filter by gene_type == "protein_coding"
protein_coding = mart[mart['gene_type'] == 'protein_coding']

# remove specified columns
columns_to_remove = ['gene_synonym', 'transcript_stable_id', 'protein_stable_id', 'gene_stable_id_version', 'protein_stable_id_version', 'transcript_stable_id_version']
protein_coding = protein_coding.drop(columns_to_remove, axis=1)

# get unique rows
protein_coding = protein_coding.drop_duplicates()

# filter out rows with missing ncbi_gene_id
protein_coding = protein_coding.dropna(subset=['ncbi_gene_id'])
protein_coding['ncbi_gene_id'] = protein_coding['ncbi_gene_id'].astype(int)

# group by 'gene_stable_id' and concatenate unique values with comma separator
protein_coding = protein_coding.groupby('gene_stable_id').apply(lambda x: pd.Series({col: ','.join(x[col].unique().astype(str)) for col in x.columns}))

# reset the index to make 'gene_stable_id' a regular column
protein_coding = protein_coding.reset_index(drop = True)

# get Ensembl list
protein_coding_ensembl = protein_coding.gene_stable_id
print("Number of Protein Coding Genes: ", len(protein_coding_ensembl))

Number of Protein Coding Genes:  22063


# Read scRNA-seq Data

The CZ CELLxGENE Discover Census provides efficient computational tooling to access, query, and analyze all single-cell RNA data from [CZ CELLxGENE Discover](https://cellxgene.cziscience.com/). First, read data parsed by `2_CELLxGENE_chunk_by_group.py`.

In [5]:
# list all matrix files
all_files = os.listdir(cellxgene_data / 'chunks')
mtx_files = [x for x in all_files if x.endswith('_matrix.csv')]
mtx_files = sorted(mtx_files)

# list all h5ad AnnData files
h5ad_files = [x for x in all_files if x.endswith('_adata.h5ad')]
h5ad_files = sorted(h5ad_files)

# get unique group IDs
group_ids = ['_'.join(x.split('_')[0:3]) for x in mtx_files]
group_ids = list(set(group_ids))

# read nervous metadata
nervous_groups = pd.read_csv(cellxgene_data / 'nervous_groups.csv')

# list number of subset cells
total_cells = nervous_groups.cell_counts.sum()
print("Total Cells: ", total_cells)

Total Cells:  9140274


Get number of chunks per dataset.

In [6]:
# iterate through groups
for index, row in nervous_groups.iterrows():
    
    # get group ID
    group_id = row['group_id']
    
    # get all files for group
    chunk_files = [x for x in h5ad_files if x.startswith(group_id)]
    chunk_files = sorted(chunk_files)
    n_chunks = len(chunk_files)

    # assign to column of nervous groups
    nervous_groups.loc[index, 'number_chunks'] = n_chunks

# coerce to integer
nervous_groups['number_chunks'] = nervous_groups['number_chunks'].astype(int)

# order by increasing cell count
nervous_groups = nervous_groups.sort_values(by="cell_counts", ascending=False)
nervous_groups.head()

,tissue,tissue_ontology_term_id,cell_type,cell_type_ontology_term_id,dataset_id,cell_counts,soma_joinid,collection_id,collection_name,collection_doi,dataset_title,dataset_h5ad_path,dataset_total_cell_count,group_id,number_chunks
0,telencephalon,UBERON:0001893,glutamatergic neuron,CL:0000679,f7c1c579-2dc0-47e2-ba19-8165c5a0e353,1258818,452,c114c20f-1ef4-49a5-9c2e-d965787fb90c,A human cell atlas of fetal gene expression,10.1126/science.aba7721,Survey of human embryonic development,f7c1c579-2dc0-47e2-ba19-8165c5a0e353.h5ad,4062980,0001893_0000679_f7c1c579-2dc0-47e2-ba19-8165c5...,6
1,middle temporal gyrus,UBERON:0002771,L2/3-6 intratelencephalic projecting glutamate...,CL:4023040,c2876b1b-06d8-4d96-a56b-5304f815b99a,570917,22,1ca90a2d-2943-483d-b678-b809bf464c30,SEA-AD: Seattle Alzheimer’s Disease Brain Cell...,NaN,Whole Taxonomy - MTG: Seattle Alzheimer's Dise...,c2876b1b-06d8-4d96-a56b-5304f815b99a.h5ad,1226855,0002771_4023040_c2876b1b-06d8-4d96-a56b-5304f8...,29
2,telencephalon,UBERON:0001893,GABAergic neuron,CL:0000617,f7c1c579-2dc0-47e2-ba19-8165c5a0e353,350759,452,c114c20f-1ef4-49a5-9c2e-d965787fb90c,A human cell atlas of fetal gene expression,10.1126/science.aba7721,Survey of human embryonic development,f7c1c579-2dc0-47e2-ba19-8165c5a0e353.h5ad,4062980,0001893_0000617_f7c1c579-2dc0-47e2-ba19-8165c5...,2
3,cerebellum,UBERON:0002037,Purkinje cell,CL:0000121,f7c1c579-2dc0-47e2-ba19-8165c5a0e353,297519,452,c114c20f-1ef4-49a5-9c2e-d965787fb90c,A human cell atlas of fetal gene expression,10.1126/science.aba7721,Survey of human embryonic development,f7c1c579-2dc0-47e2-ba19-8165c5a0e353.h5ad,4062980,0002037_0000121_f7c1c579-2dc0-47e2-ba19-8165c5...,3
4,cerebellum,UBERON:0002037,astrocyte,CL:0000127,f7c1c579-2dc0-47e2-ba19-8165c5a0e353,289917,452,c114c20f-1ef4-49a5-9c2e-d965787fb90c,A human cell atlas of fetal gene expression,10.1126/science.aba7721,Survey of human embryonic development,f7c1c579-2dc0-47e2-ba19-8165c5a0e353.h5ad,4062980,0002037_0000127_f7c1c579-2dc0-47e2-ba19-8165c5...,2


Group by dataset to iterate across datasets.

In [7]:
# get dataset level information
dataset_columns = ['dataset_id', 'soma_joinid', 'collection_id', 'collection_name', 'collection_doi',
                  'dataset_title', 'dataset_h5ad_path', 'dataset_total_cell_count']
nervous_datasets = nervous_groups.loc[:, dataset_columns]
nervous_datasets = nervous_datasets.drop_duplicates()

# define the function to concatenate unique values
def grp(x):
    return ', '.join(sorted(pd.unique(x)))

# get aggregated metadata
nervous_datasets_agg = nervous_groups.groupby('dataset_id').agg(
    tissue=('tissue', grp),
    tissue_ontology_term_id=('tissue_ontology_term_id', grp),
    cell_type=('cell_type', grp),
    cell_type_ontology_term_id=('cell_type_ontology_term_id', grp),
    cell_counts=('cell_counts', 'sum'),
    number_chunks=('number_chunks', 'sum'),
    number_tissue=('tissue_ontology_term_id', 'nunique'),
    number_cell_types=('cell_type_ontology_term_id', 'nunique')
).reset_index()

# merge nervous data
nervous_datasets = nervous_datasets.merge(nervous_datasets_agg, on="dataset_id")
nervous_datasets = nervous_datasets.sort_values(by="cell_counts", ascending=False)
nervous_datasets.head()

,dataset_id,soma_joinid,collection_id,collection_name,collection_doi,dataset_title,dataset_h5ad_path,dataset_total_cell_count,tissue,tissue_ontology_term_id,cell_type,cell_type_ontology_term_id,cell_counts,number_chunks,number_tissue,number_cell_types
0,f7c1c579-2dc0-47e2-ba19-8165c5a0e353,452,c114c20f-1ef4-49a5-9c2e-d965787fb90c,A human cell atlas of fetal gene expression,10.1126/science.aba7721,Survey of human embryonic development,f7c1c579-2dc0-47e2-ba19-8165c5a0e353.h5ad,4062980,"cerebellum, telencephalon","UBERON:0001893, UBERON:0002037","GABAergic neuron, Purkinje cell, astrocyte, en...","CL:0000120, CL:0000121, CL:0000122, CL:0000123...",2843210,29,2,15
1,c2876b1b-06d8-4d96-a56b-5304f815b99a,22,1ca90a2d-2943-483d-b678-b809bf464c30,SEA-AD: Seattle Alzheimer’s Disease Brain Cell...,NaN,Whole Taxonomy - MTG: Seattle Alzheimer's Dise...,c2876b1b-06d8-4d96-a56b-5304f815b99a.h5ad,1226855,middle temporal gyrus,UBERON:0002771,L2/3-6 intratelencephalic projecting glutamate...,"CL:0000128, CL:0000129, CL:0002453, CL:0002605...",1149751,56,1,18
2,56c4912d-2bae-4b64-98f2-af8a84389208,285,999f2a15-3d7e-440b-96ae-2c806799c08c,"Harmonized single-cell landscape, intercellula...",10.1101/2022.08.27.505439,Extended GBmap,56c4912d-2bae-4b64-98f2-af8a84389208.h5ad,1135677,"basal ganglion, brain, forebrain, frontal lobe...","UBERON:0000955, UBERON:0001871, UBERON:0001872...","B cell, dendritic cell, endothelial cell, macr...","CL:0000115, CL:0000128, CL:0000129, CL:0000235...",1063427,74,13,15
7,ae4f8ddd-cac9-4172-9681-2175da462f2e,415,c8565c6a-01a1-435b-a549-f11b452a83a8,Human developing neocortex by area,10.1038/s41586-021-03910-8,Second Trimester Human Developing Brain Region...,ae4f8ddd-cac9-4172-9681-2175da462f2e.h5ad,457965,"parietal cortex, prefrontal cortex, primary mo...","UBERON:0000451, UBERON:0001384, UBERON:0002436...","cerebral cortex GABAergic interneuron, forebra...","CL:0000003, CL:0000129, CL:0000679, CL:0002453...",448831,35,6,7
3,f32c2c13-bb1a-4ffd-a457-60b64ecfa4cb,115,283d65eb-dd53-496d-adb7-7570c7caa443,Transcriptomic diversity of cell types across ...,10.1101/2022.10.12.511898,Dissection: Cerebral cortex (Cx) - Precentral ...,f32c2c13-bb1a-4ffd-a457-60b64ecfa4cb.h5ad,116576,cerebral cortex,UBERON:0000956,"astrocyte, neuron, oligodendrocyte","CL:0000127, CL:0000128, CL:0000540",115594,7,1,3


Iterate through (`tissue`, `cell_type`, `dataset_id`) combinations in order of decreasing total cell count.

In [18]:
# sum cell_counts for datasets where number_cell_types == 1
single_type_sum = nervous_datasets.loc[nervous_datasets['number_cell_types'] == 1, 'cell_counts'].sum()
print("Cells from datasets with only 1 cell type: ", single_type_sum)
print("Cells from datasets with > 1 cell type: ", nervous_datasets.cell_counts.sum() - single_type_sum)

Cells from datasets with only 1 cell type:  386656
Cells from datasets with > 1 cell type:  8753618


In [21]:
# iterate through datasets
# for index, row in nervous_datasets.iterrows():

# for example:
index = 0
row = nervous_datasets.iloc[index]

### DATASET ID
# Get information per dataset
dataset_id_i = row['dataset_id']
tissues = pd.unique(nervous_groups[nervous_groups['dataset_id'] == dataset_id_i]['tissue'])
cell_types = pd.unique(nervous_groups[nervous_groups['dataset_id'] == dataset_id_i]['cell_type'])
group_ids = pd.unique(nervous_groups[nervous_groups['dataset_id'] == dataset_id_i]['group_id'])

# Dataset-level statistics
n_cells = row['cell_counts']
n_chunks = row['number_chunks']
n_tissue = row['number_tissue']

# Message
start_time = time.time()
print("\n\nANALYSIS {}/{}:".format(index + 1, len(nervous_datasets)))
print("Starting analysis {} out of {} at {}".format(index + 1, len(nervous_datasets), time.strftime("%H:%M on %m/%d.", time.localtime(start_time))))
print(" - Tissue: {}\n - Cell Type: {}\n - Dataset ID: {}\n - Cell Count: {}".format(tissues, cell_types, dataset_id_i, n_cells))



ANALYSIS 1/122:
Starting analysis 1 out of 122 at 18:40 on 07/11.
 - Tissue: ['telencephalon' 'cerebellum']
 - Cell Type: ['glutamatergic neuron' 'GABAergic neuron' 'Purkinje cell' 'astrocyte'
 'inhibitory interneuron' 'granule cell'
 'neuron associated cell (sensu Vertebrata)' 'neuron' 'stellate neuron'
 'neuronal brush cell' 'glial cell' 'macroglial cell' 'oligodendrocyte'
 'endothelial cell of vascular tree' 'microglial cell']
 - Dataset ID: f7c1c579-2dc0-47e2-ba19-8165c5a0e353
 - Cell Count: 2843210


In [22]:
# define list to store groups
group_list = []
group_var_list = []
group_cell_counts = []

# iterate over groups, read data, and concatenate
for group_index, group_id_i in enumerate(group_ids):

    # for example
    # group_id_i = group_ids[0]

    # print progress
    print("Reading group {} out of {}.".format(group_index + 1, len(group_ids)))

    # get list of chunks
    chunk_files = [x for x in h5ad_files if x.startswith(group_id_i)]

    # read var file with gene metadata
    group_var = pd.read_csv(cellxgene_data / 'chunks' / (group_id_i + '_var.csv'))
    group_var['soma_joinid'] = group_var['soma_joinid'].astype(str)
    group_var = group_var.set_index('soma_joinid')
    group_var_list.append(group_var)

    # define list to store chunks
    # chunk_list = []

    # iterate through chunks
    for chunk_index, chunk_file in enumerate(chunk_files):

        # read chunk files with expression values and cell metadata
        print("Chunks read: {} out of {}.".format(chunk_index + 1, len(chunk_files)), end="\r")
        chunk_adata = sc.read_h5ad(cellxgene_data / 'chunks' / chunk_file)
        # chunk_adata.var = group_var

        # append to chunk list
        # chunk_list.append(chunk_adata)
        group_list.append(chunk_adata)
        group_cell_counts.append(chunk_adata.shape[0])

    # concatenate all chunks into one AnnData object
    # group_adata = anndata.concat(chunk_list, axis=0, label='chunk_id')
    # group_adata.var = group_var

adata = anndata.concat(group_list, axis=0, label='chunk_id')
# adata.var = group_var

Reading group 1 out of 20.
Reading group 2 out of 20.
Reading group 3 out of 20.
Reading group 4 out of 20.
Reading group 5 out of 20.
Reading group 6 out of 20.
Reading group 7 out of 20.
Reading group 8 out of 20.
Reading group 9 out of 20.
Reading group 10 out of 20.
Reading group 11 out of 20.
Reading group 12 out of 20.
Reading group 13 out of 20.
Reading group 14 out of 20.
Reading group 15 out of 20.
Reading group 16 out of 20.
Reading group 17 out of 20.
Reading group 18 out of 20.
Reading group 19 out of 20.
Reading group 20 out of 20.


/n/data1/hms/dbmi/zitnik/lab/users/an252/NeuroKG/neuroKG/neuroKG_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [118]:
# merge AnnData
adata = anndata.concat(group_list, axis=0, label='chunk_id')

# get duplicated cells
dup_cells = adata.obs.loc[adata.obs.index[adata.obs.index.duplicated()]]

# make obs names unique
adata.obs_names_make_unique() # adds -1 to each

# fix duplicates
for dup_cell in dup_cells.index.unique():

    # get duplicate name
    dup_name = dup_cell + '-1'

    # replace duplicate values
    adata[dup_cell, ].X = adata[dup_cell, ].X + adata[dup_name, ].X

/n/data1/hms/dbmi/zitnik/lab/users/an252/NeuroKG/neuroKG/neuroKG_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


<1x60664 sparse matrix of type '<class 'numpy.int64'>'
	with 282 stored elements in Compressed Sparse Row format>

In [114]:
adata['47566848-1', ].X

<1x60664 sparse matrix of type '<class 'numpy.int64'>'
	with 87 stored elements in Compressed Sparse Row format>

In [27]:
# get duplicated cells
dup_cells = adata.obs.loc[adata.obs.index[adata.obs.index.duplicated()]]
tmp = np.array(adata[dup_cells.index[0], ].X.todense())
zero_coords = np.nonzero(tmp)
zero_rows = zero_coords[0]
zero_cols = zero_coords[1]

In [107]:
dup_cells

,dataset_id,assay,assay_ontology_term_id,cell_type,cell_type_ontology_term_id,development_stage,development_stage_ontology_term_id,disease,disease_ontology_term_id,donor_id,...,self_reported_ethnicity,self_reported_ethnicity_ontology_term_id,sex,sex_ontology_term_id,suspension_type,tissue,tissue_ontology_term_id,tissue_general,tissue_general_ontology_term_id,chunk_id
soma_joinid,,,,,,,,,,,,,,,,,,,,,
47566848,f7c1c579-2dc0-47e2-ba19-8165c5a0e353,sci-RNA-seq,EFO:0010550,glutamatergic neuron,CL:0000679,16th week post-fertilization human stage,HsapDv:0000053,normal,PATO:0000461,H27058,...,unknown,unknown,female,PATO:0000383,nucleus,telencephalon,UBERON:0001893,brain,UBERON:0000955,0
47566848,f7c1c579-2dc0-47e2-ba19-8165c5a0e353,sci-RNA-seq,EFO:0010550,glutamatergic neuron,CL:0000679,16th week post-fertilization human stage,HsapDv:0000053,normal,PATO:0000461,H27058,...,unknown,unknown,female,PATO:0000383,nucleus,telencephalon,UBERON:0001893,brain,UBERON:0000955,1
47566849,f7c1c579-2dc0-47e2-ba19-8165c5a0e353,sci-RNA-seq,EFO:0010550,glutamatergic neuron,CL:0000679,16th week post-fertilization human stage,HsapDv:0000053,normal,PATO:0000461,H27058,...,unknown,unknown,female,PATO:0000383,nucleus,telencephalon,UBERON:0001893,brain,UBERON:0000955,0
47566849,f7c1c579-2dc0-47e2-ba19-8165c5a0e353,sci-RNA-seq,EFO:0010550,glutamatergic neuron,CL:0000679,16th week post-fertilization human stage,HsapDv:0000053,normal,PATO:0000461,H27058,...,unknown,unknown,female,PATO:0000383,nucleus,telencephalon,UBERON:0001893,brain,UBERON:0000955,1
47566850,f7c1c579-2dc0-47e2-ba19-8165c5a0e353,sci-RNA-seq,EFO:0010550,glutamatergic neuron,CL:0000679,12th week post-fertilization human stage,HsapDv:0000049,normal,PATO:0000461,H27474,...,unknown,unknown,male,PATO:0000384,nucleus,telencephalon,UBERON:0001893,brain,UBERON:0000955,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47054845,f7c1c579-2dc0-47e2-ba19-8165c5a0e353,sci-RNA-seq,EFO:0010550,astrocyte,CL:0000127,17th week post-fertilization human stage,HsapDv:0000054,normal,PATO:0000461,H27477,...,unknown,unknown,male,PATO:0000384,nucleus,cerebellum,UBERON:0002037,brain,UBERON:0000955,12
47054846,f7c1c579-2dc0-47e2-ba19-8165c5a0e353,sci-RNA-seq,EFO:0010550,astrocyte,CL:0000127,17th week post-fertilization human stage,HsapDv:0000054,normal,PATO:0000461,H27477,...,unknown,unknown,male,PATO:0000384,nucleus,cerebellum,UBERON:0002037,brain,UBERON:0000955,11
47054846,f7c1c579-2dc0-47e2-ba19-8165c5a0e353,sci-RNA-seq,EFO:0010550,astrocyte,CL:0000127,17th week post-fertilization human stage,HsapDv:0000054,normal,PATO:0000461,H27477,...,unknown,unknown,male,PATO:0000384,nucleus,cerebellum,UBERON:0002037,brain,UBERON:0000955,12


In [93]:
test_data_1 = np.random.rand(2, 3)
test_1 = anndata.AnnData(test_data_1)

In [96]:
adata.X

<2852408x60664 sparse matrix of type '<class 'numpy.int64'>'
	with 1830797084 stored elements in Compressed Sparse Row format>